# 🚀 NIDS-ML Phase 1: Hyperparameter Tuning (Optimized)

Questo notebook esegue il tuning degli iperparametri in modo efficiente su Kaggle.

### ✨ Caratteristiche
- **Git Sparse Checkout**: Scarica solo il codice sorgente necessario (`src/`) senza clonare l'intera cronologia o file inutili.
- **Smart Dataset Linking**: Usa Symlinks invece di copiare i dati, risparmiando spazio e tempo di avvio.
- **Single Model Focus**: Ottimizza un modello alla volta per massimizzare l'utilizzo delle 12 ore di runtime di Kaggle.
- **Robust Output**: Salva e zippa automaticamente i risultati per il download.

In [ ]:
# ============================================================================
# 1. CONFIGURAZIONE PARAMETRI
# ============================================================================

# --- Selezione Modello ---
# Scegli UNO solo per sessione per evitare timeout (random_forest, xgboost, lightgbm)
TARGET_MODEL = 'xgboost'

# --- Strategia di Tuning ---
TUNING_METHOD = 'bayesian'  # 'random' o 'bayesian'

# Parametri Ricerca
N_ITER   = 50      # Usato solo se method='random' (numero totale combinazioni)
N_TRIALS = 100     # Usato solo se method='bayesian' (numero tentativi Optuna)
TIMEOUT_HOURS = 11 # Timeout sicurezza (Kaggle killa a 12h)

# Validazione
CV_FOLDS = 5       # Cross-Validation folds

# --- Repo Config ---
REPO_URL = "https://github.com/Riiccardob/NIDS-ML-SSR2"
BRANCH = "main"

print(f"🎯 Target: {TARGET_MODEL} ({TUNING_METHOD})")
print(f"⏱️  Timeout: {TIMEOUT_HOURS}h")

In [ ]:
# ============================================================================
# 2. SETUP AMBIENTE & GIT SPARSE CHECKOUT
# ============================================================================

import os
import sys
from pathlib import Path
import shutil

# Rilevamento Ambiente
KAGGLE_ENV = Path("/kaggle/input").exists()
PROJECT_ROOT = Path("/kaggle/working") if KAGGLE_ENV else Path.cwd()

if KAGGLE_ENV:
    print("☁️  Running on Kaggle Environment")
    
    # Pulizia preventiva se necessario
    if (PROJECT_ROOT / "src").exists():
        shutil.rmtree(PROJECT_ROOT / "src")
    if (PROJECT_ROOT / "temp_repo").exists():
        shutil.rmtree(PROJECT_ROOT / "temp_repo")

    # --- GIT SPARSE CHECKOUT (Opzione B) ---
    print(f"⬇️  Cloning source code from {REPO_URL}...")
    # 1. Clone 'vuoto' (senza file, solo metadata minimi)
    !git clone --filter=blob:none --no-checkout {REPO_URL} temp_repo
    
    # 2. Configura Sparse Checkout per scaricare SOLO 'src' e 'requirements.txt'
    %cd temp_repo
    !git sparse-checkout init --cone
    !git sparse-checkout set src requirements.txt
    
    # 3. Checkout del branch
    !git checkout {BRANCH}
    
    # 4. Sposta i file nella root di lavoro e pulisci
    %cd ..
    !mv temp_repo/src .
    !mv temp_repo/requirements.txt .
    !rm -rf temp_repo
    
    print("✅ Code setup complete (src/ downloaded).")
    
    # Installazione dipendenze
    print("📦 Installing dependencies...")
    !pip install -q -r requirements.txt
    !pip install -q optuna  # Necessario per bayesian
    
else:
    print("💻 Running Local Environment")
    # Assumiamo che in locale siamo già nella root del progetto
    if not (PROJECT_ROOT / "src").exists():
        PROJECT_ROOT = PROJECT_ROOT.parent # Risale di un livello se siamo in 'notebooks/'

sys.path.insert(0, str(PROJECT_ROOT))
os.chdir(PROJECT_ROOT)

# Creazione cartelle output
for d in ["data/raw", "data/processed", "artifacts", "tuning_results", "logs"]:
    (PROJECT_ROOT / d).mkdir(parents=True, exist_ok=True)

In [ ]:
# ============================================================================
# 3. DATASET SETUP (SYMLINKS)
# ============================================================================

raw_dir = PROJECT_ROOT / "data" / "raw"
count = 0

if KAGGLE_ENV:
    print("🔗 Creating Symlinks for Dataset...")
    input_dir = Path("/kaggle/input")
    
    # Cerca ricorsivamente file CSV o Parquet negli input di Kaggle
    for p in input_dir.glob("**/*"):
        if p.is_file() and p.suffix in ['.csv', '.parquet']:
            dest = raw_dir / p.name
            if not dest.exists():
                try:
                    os.symlink(p, dest)
                    print(f"  -> Linked: {p.name}")
                    count += 1
                except Exception as e:
                    print(f"  ❌ Error linking {p.name}: {e}")
else:
    # In locale conta solo i file esistenti
    count = len(list(raw_dir.glob("*.csv"))) + len(list(raw_dir.glob("*.parquet")))

print(f"\n✅ Dataset pronto: {count} file disponibili in data/raw/")

In [ ]:
# ============================================================================
# 4. PREPROCESSING
# ============================================================================

# Controlla se il preprocessing è già stato fatto per risparmiare tempo in caso di riavvio sessione
if not (PROJECT_ROOT / "data" / "processed" / "test.parquet").exists():
    print("⚙️  Avvio Preprocessing...")
    # Parametri: balance-ratio 2.0 per ridurre overfitting su classi maggioritarie
    !python src/preprocessing.py --balance-ratio 2.0 --n-jobs -1
else:
    print("⏩ Dati processati trovati. Skipping preprocessing.")

In [ ]:
# ============================================================================
# 5. FEATURE ENGINEERING
# ============================================================================

if not (PROJECT_ROOT / "artifacts" / "selected_features.json").exists():
    print("🧪 Avvio Feature Engineering...")
    !python src/feature_engineering.py --n-features 30 --n-jobs -1
else:
    print("⏩ Artifacts feature engineering trovati. Skipping.")

In [ ]:
# ============================================================================
# 6. HYPERPARAMETER TUNING
# ============================================================================

print(f"🔥 START TUNING: {TARGET_MODEL.upper()} using {TUNING_METHOD.upper()}")

# Costruzione comando dinamico
cmd = f"python src/hyperparameter_tuning.py --model {TARGET_MODEL} --cv {CV_FOLDS} --task binary --n-jobs -1"

if TUNING_METHOD == 'bayesian':
    # Converte ore in secondi per lo script
    timeout_sec = int(TIMEOUT_HOURS * 3600)
    cmd += f" --method bayesian --n-trials {N_TRIALS} --timeout {timeout_sec}"
else:
    cmd += f" --method random --n-iter {N_ITER}"

print(f"Running command: {cmd}\n")

# Esecuzione
!{cmd}

In [ ]:
# ============================================================================
# 7. VERIFICA RISULTATI E PACKAGING
# ============================================================================

import json
import glob

print("📊 Checking Results...")

# Cerca l'ultimo file JSON generato per il modello target
results_path = PROJECT_ROOT / "tuning_results" / TARGET_MODEL
json_files = list(results_path.glob("*.json"))

if json_files:
    # Prende il più recente
    latest_file = max(json_files, key=os.path.getctime)
    
    with open(latest_file, 'r') as f:
        data = json.load(f)
        
    print(f"\n🏆 BEST RESULT FOUND ({latest_file.name}):")
    print(f"   Score: {data.get('best_score', 'N/A'):.4f}")
    print(f"   Best Params: {json.dumps(data.get('best_params', {}), indent=2)}")
else:
    print("⚠️ Nessun file di risultati trovato. Il tuning potrebbe essere fallito.")

print("\n📦 Packaging results for download...")
# Zippa cartelle tuning_results e logs per facile download da Kaggle
!zip -r phase1_results.zip tuning_results logs artifacts > /dev/null
print("✅ Created 'phase1_results.zip'. Download this file from the Output tab.")